In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.7 MB/s eta 0:00:00


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.1.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 19.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import pandas as pd
import numpy as np
import torch

In [4]:

apply_train = pd.read_csv('apply_train.csv')
print(len(apply_train))
apply_train.head()


57946


,resume_seq,recruitment_seq
0,U05833,R03838
1,U06456,R02144
2,U07807,R01877
3,U04842,R02463
4,U08336,R00112


In [5]:

resume = pd.read_csv('resume.csv')
print(len(resume))
resume.head()


8482


,resume_seq,reg_date,updated_date,degree,graduate_date,hope_salary,last_salary,text_keyword,job_code_seq1,job_code_seq2,job_code_seq3,career_month,career_job_code
0,U00606,2020-03-04,2020-05-22,4,2008,3500.0,3500.0,스타일디자이너;우븐디자이너,재료·화학·섬유·의복,NaN,NaN,67,NaN
1,U00509,2019-08-25,2020-09-02,2,0,0.0,3700.0,상품기획;MD;기획;머천다이저;머천다이징,재료·화학·섬유·의복,NaN,NaN,84,섬유;봉제;가방;의류
2,U02012,2017-11-20,2020-01-26,5,1979,3500.0,3100.0,니트디자인,재료·화학·섬유·의복,NaN,NaN,121,학교;학원;직원훈련(교육서비스)
3,U04599,2020-05-13,2020-05-28,4,2012,0.0,2500.0,MD;기획MD,재료·화학·섬유·의복,NaN,NaN,24,섬유;봉제;가방;의류
4,U07573,2019-07-23,2020-03-08,4,2010,1900.0,0.0,디자이너;남성복;스포츠웨어;편집디자인;코디네이터;일러스트레이터;VMD;MD,재료·화학·섬유·의복,NaN,NaN,0,NaN


In [6]:

new_resume = resume[['resume_seq','text_keyword']]
new_resume = new_resume.sort_values(by='resume_seq')
new_resume = new_resume.reset_index(drop=True)
new_resume['text_keyword'].fillna('empty', inplace=True)
new_resume.head()


,resume_seq,text_keyword
0,U00001,디자이너
1,U00002,디자이너
2,U00003,남성복디자이너;TD캐주얼
3,U00004,상품기획;영업기획
4,U00005,인사;총무;경영;MD;상품기획


In [7]:

recruitment = pd.read_csv('recruitment.csv')
print(len(recruitment))
recruitment.head()


6695


,recruitment_seq,address_seq1,address_seq2,address_seq3,career_end,career_start,check_box_keyword,education,major_task,qualifications,text_keyword
0,R02264,3.0,NaN,NaN,0,0,2507;2707;2810,4,8,1,NaN
1,R06317,3.0,NaN,NaN,0,0,2204;2205;2707,3,2,1,NaN
2,R04017,3.0,NaN,NaN,0,0,2101;2108;2201;2707,3,2,1,NaN
3,R02865,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,1,NaN
4,R04890,3.0,NaN,NaN,0,0,2201;2204;2205;2707,2,2,2,NaN


In [8]:

new_recruitment = recruitment[['recruitment_seq','text_keyword']]
new_recruitment = new_recruitment.sort_values(by='recruitment_seq')
new_recruitment = new_recruitment.reset_index(drop=True)
new_recruitment['text_keyword'].fillna('empty', inplace=True)
new_recruitment.head()


,recruitment_seq,text_keyword
0,R00001,empty
1,R00002,empty
2,R00003,empty
3,R00004,empty
4,R00005,empty


In [9]:

## Getting node features?


import torch
from transformers import BertTokenizer, BertModel
from collections import defaultdict

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

word = pd.concat([new_resume['text_keyword'], new_recruitment['text_keyword']], ignore_index=True,join='inner')
new_word = pd.DataFrame()
new_word['text_keyword'] = word


In [10]:


# defaultdict를 사용하여 리스트를 기본값으로 설정
if 'tokenized_text' in new_word.columns:
        new_word.drop('tokenized_text', axis=1, inplace=True)


In [11]:
my_dict = defaultdict(list)
new_word['tokenized_text']= None


In [12]:
new_word

,text_keyword,tokenized_text
0,디자이너,None
1,디자이너,None
2,남성복디자이너;TD캐주얼,None
3,상품기획;영업기획,None
4,인사;총무;경영;MD;상품기획,None
...,...,...
15172,empty,None
15173,empty,None
15174,empty,None
15175,empty,None


In [13]:

for index, element in enumerate(word):
    text = element.split(';')
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    new_word.at[index,'tokenized_text'] = tokens

new_word['bert_embedding']=None


In [14]:
new_word

,text_keyword,tokenized_text,bert_embedding
0,디자이너,"[input_ids, token_type_ids, attention_mask]",None
1,디자이너,"[input_ids, token_type_ids, attention_mask]",None
2,남성복디자이너;TD캐주얼,"[input_ids, token_type_ids, attention_mask]",None
3,상품기획;영업기획,"[input_ids, token_type_ids, attention_mask]",None
4,인사;총무;경영;MD;상품기획,"[input_ids, token_type_ids, attention_mask]",None
...,...,...,...
15172,empty,"[input_ids, token_type_ids, attention_mask]",None
15173,empty,"[input_ids, token_type_ids, attention_mask]",None
15174,empty,"[input_ids, token_type_ids, attention_mask]",None
15175,empty,"[input_ids, token_type_ids, attention_mask]",None


In [15]:

# Check if a GPU is available, and if so, use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [16]:
device

device(type='cuda')

In [17]:

features = torch.FloatTensor(1, 10, 768).to(device)
# Move your BERT model to the GPU
model.to(device)

# Assuming new_word is a DataFrame or some iterable containing tokenized texts
with torch.no_grad():
    for index, row in enumerate(new_word['tokenized_text']):
        if index % 1000 ==0:
            print(index)
        tokens = new_word.at[index, 'tokenized_text']

        # You need to wrap tokens in a dictionary and specify the input format required by the model
        input_dict = {
            'input_ids': tokens['input_ids'].to(device),       # Input token IDs
            'attention_mask': tokens['attention_mask'].to(device)  # Attention mask
        }

        output = model(**input_dict)
        embeddings = output.last_hidden_state


        # Assuming new_word is a DataFrame, you can store the embeddings in a new column
        new_word.at[index, 'bert_embedding'] = embeddings


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000


In [18]:

new_resume['bert'] = new_word['bert_embedding'][:8482]
#new_recruitment= new_recruitment.drop(columns=['bert'])
new_recruitment['bert'] = new_word['bert_embedding'][8482:].reset_index(drop=True)
print(len(new_resume),len(new_recruitment))

8482 6695


In [19]:

node_relation = apply_train.sort_values(by='resume_seq')
node_relation = node_relation.reset_index(drop=True)
node_relation.head()

for index in range(len(node_relation)):
    node_relation.at[index,'resume_seq']=int(node_relation.at[index,'resume_seq'][-5:])-1
    node_relation.at[index,'recruitment_seq']=int(node_relation.at[index,'recruitment_seq'][-5:])-1

resume_id = [i for i in node_relation['resume_seq']]
temp1= torch.from_numpy(np.array(resume_id))
recruit_id = [i for i in node_relation['recruitment_seq']]
temp2= torch.from_numpy(np.array(recruit_id))
edge_index = torch.stack([temp1,temp2], dim=0)


In [20]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

temp = new_resume['bert'][0]
temp = []
for i in range(len(new_resume)):
    batch_size, max_seq_length, embedding_dim = new_resume['bert'][i].size()
    mean_pooled_embeddings =new_resume['bert'][i].mean(dim=1)
    final_representation = mean_pooled_embeddings.mean(dim=0)
    temp.append(final_representation)
resume_features = torch.stack(temp)
print(resume_features.shape)

temp = []
for i in range(len(new_recruitment)):
    batch_size, max_seq_length, embedding_dim = new_recruitment['bert'][i].size()

    # Mean pooling along the sequence length dimension
    mean_pooled_embeddings =new_recruitment['bert'][i].mean(dim=1)

    # Now mean_pooled_embeddings is a tensor of shape [batch_size, embedding_dim]
    # If you want a single vector representation, you can further average along the batch dimension
    final_representation = mean_pooled_embeddings.mean(dim=0)
    temp.append(final_representation)
recruitment_features = torch.stack(temp)
print(recruitment_features.shape)


torch.Size([8482, 768])
torch.Size([6695, 768])


In [21]:

data = HeteroData()
data["resume"].node_id = torch.arange(len(new_resume))
data["recruitment"].node_id = torch.arange(len(new_recruitment))
data["resume"].x = resume_features
data['recruitment'].x = recruitment_features
data["resume", "apply", "recruitment"].edge_index = edge_index
data = T.ToUndirected()(data)
print(data.metadata())



(['resume', 'recruitment'], [('resume', 'apply', 'recruitment'), ('recruitment', 'rev_apply', 'resume')])


In [22]:

transform = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("resume", "apply", "recruitment"),
    rev_edge_types=("recruitment", "rev_apply", "resume"),
)
train_data, val_data, test_data = transform(data)



In [23]:
data

HeteroData(
  resume={
    node_id=[8482],
    x=[8482, 768],
  },
  recruitment={
    node_id=[6695],
    x=[6695, 768],
  },
  (resume, apply, recruitment)={ edge_index=[2, 57946] },
  (recruitment, rev_apply, resume)={ edge_index=[2, 57946] }
)

In [24]:
train_data

HeteroData(
  resume={
    node_id=[8482],
    x=[8482, 768],
  },
  recruitment={
    node_id=[6695],
    x=[6695, 768],
  },
  (resume, apply, recruitment)={
    edge_index=[2, 32451],
    edge_label=[13907],
    edge_label_index=[2, 13907],
  },
  (recruitment, rev_apply, resume)={ edge_index=[2, 32451] }
)

In [25]:

from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
edge_label_index = train_data["resume", "apply", "recruitment"].edge_label_index
edge_label = train_data["resume", "apply", "recruitment"].edge_label
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=2.0,
    edge_label_index=(("resume", "apply", "recruitment"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)




/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [28]:

from torch import Tensor
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x
# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_resume: Tensor, x_recruit: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_resume = x_resume[edge_label_index[0]]
        edge_feat_recruit = x_recruit[edge_label_index[1]]
        # Apply dot-product to get a prediction per supervision edge:
        edge_predictions = (edge_feat_resume * edge_feat_recruit).sum(dim=-1) # edge score
        return edge_predictions

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.recruit_lin = torch.nn.Linear(768, hidden_channels)
        self.resume_emb = torch.nn.Embedding(data["resume"].num_nodes, hidden_channels)
        self.recruit_emb = torch.nn.Embedding(data["recruitment"].num_nodes, hidden_channels)
        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)
        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())
        self.classifier = Classifier()
    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "resume": self.recruit_lin(data["resume"].x)+self.resume_emb(data["resume"].node_id),
          "recruitment": self.recruit_lin(data["recruitment"].x) + self.recruit_emb(data["recruitment"].node_id),
        }
        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["resume"],
            x_dict["recruitment"],
            data["resume", "apply", "recruitment"].edge_label_index,
        )
        return pred
    def get_node_embedding(self, data: HeteroData) -> Tensor:
        x_dict = {
          "resume": self.resume_emb(data["resume"].node_id),
          "recruitment": self.recruit_lin(data["recruitment"].x) + self.recruit_emb(data["recruitment"].node_id),
        }
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        return x_dict

model = Model(hidden_channels=64).to(device)



/usr/local/lib/python3.10/dist-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


In [29]:

# import tqdm
# import torch.nn.functional as F
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# for epoch in range(1, 100):
#     total_loss = total_exampes = 0
#     for sampled_data in tqdm.tqdm(train_loader):
#         optimizer.zero_grad()
#         sampled_data.to(device)
#         pred = model(sampled_data)
#         ground_truth = sampled_data["resume", "apply", "recruitment"].edge_label
#         loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
#         loss.backward()
#         optimizer.step()
#         total_loss += float(loss) * pred.numel()
#         total_examples += pred.numel()
#     print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")


In [38]:
#Define the validation seed edges:
edge_label_index = val_data["resume", "apply", "recruitment"].edge_label_index
edge_label = val_data["resume", "apply", "recruitment"].edge_label
val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    edge_label_index=(("resume", "apply", "recruitment"), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)
sampled_data = next(iter(val_loader))

/usr/local/lib/python3.10/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "


In [ ]:
# from sklearn.metrics import roc_auc_score
# preds = []
# ground_truths = []
# for sampled_data in tqdm.tqdm(val_loader):
#     with torch.no_grad():
#         sampled_data.to(device)
#         preds.append(model(sampled_data))
#         ground_truths.append(sampled_data["resume", "apply", "recruitment"].edge_label)
# pred = torch.cat(preds, dim=0).cpu().numpy()
# ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
# auc = roc_auc_score(ground_truth, pred)
# print()
# print(f"Validation AUC: {auc:.4f}")

In [39]:
import tqdm
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=100, eta_min=0.00001)
        #dataload
# Training loop with validation evaluation
for epoch in range(1, 100):
    total_loss = total_examples = 0

    # Training phase
    model.train()
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        sampled_data.to(device)
        pred = model(sampled_data)
        ground_truth = sampled_data["resume", "apply", "recruitment"].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()

    # Validation phase
    model.eval()
    preds = []
    ground_truths = []
    with torch.no_grad():
        for sampled_data in tqdm.tqdm(val_loader):
            sampled_data.to(device)
            preds.append(model(sampled_data))
            ground_truths.append(sampled_data["resume", "apply", "recruitment"].edge_label)

    # Calculate AUC on validation set
    pred = torch.cat(preds, dim=0).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    auc = roc_auc_score(ground_truth, pred)

    # Print epoch statistics
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}, Validation AUC: {auc:.4f}")

100%|██████████| 46/46 [00:01<00:00, 28.93it/s]


Epoch: 001, Loss: 0.5778, Validation AUC: 0.7540


100%|██████████| 46/46 [00:01<00:00, 43.08it/s]


Epoch: 002, Loss: 0.5245, Validation AUC: 0.7717


100%|██████████| 46/46 [00:01<00:00, 43.68it/s]


...

Epoch: 097, Loss: 0.0663, Validation AUC: 0.8127


100%|██████████| 46/46 [00:01<00:00, 41.23it/s]


Epoch: 098, Loss: 0.0742, Validation AUC: 0.8144


100%|██████████| 46/46 [00:01<00:00, 41.01it/s]

Epoch: 099, Loss: 0.0710, Validation AUC: 0.8073


In [40]:
node_embedding = model.get_node_embedding(data.to(device))

In [41]:
node_embedding

{'resume': tensor([[-2.7470,  1.0213, -0.5231,  ...,  0.2158,  1.3338,  0.9820],
         [-2.2999,  0.9313, -0.7348,  ..., -1.2554,  0.3728,  0.3092],
         [-2.5286,  2.1024, -2.8081,  ..., -1.1794,  0.7350,  0.5356],
         ...,
         [-3.8687,  0.4649, -0.1001,  ..., -2.1462,  0.4398,  0.6851],
         [-3.7942,  1.8345,  0.8550,  ...,  0.2690,  0.9922,  1.0054],
         [-2.4863,  1.0202,  0.1684,  ..., -1.6381,  0.2508,  0.5624]],
        device='cuda:0', grad_fn=<AddBackward0>),
 'recruitment': tensor([[ 2.1999,  0.3543, -0.9561,  ..., -0.2925,  0.3092,  0.2855],
         [ 3.4856,  0.6719, -0.7798,  ..., -0.4958, -0.1023, -1.0237],
         [ 2.4421,  0.5019, -0.6673,  ...,  0.2610, -0.3893, -1.6876],
         ...,
         [ 2.8918,  0.1139, -1.8310,  ...,  0.4769, -2.7507, -0.8156],
         [ 2.9838,  0.6576,  0.3007,  ...,  0.8536, -1.0835, -0.5893],
         [ 2.3745,  0.1091, -0.3364,  ...,  2.1104, -0.7497,  0.2034]],
        device='cuda:0', grad_fn=<AddBackwa

In [42]:
node_embedding['resume'].shape

torch.Size([8482, 64])

In [43]:
node_embedding['recruitment'].shape

torch.Size([6695, 64])

In [56]:
node_relation[node_relation['resume_seq']==0]['recruitment_seq'].values

array([5209, 5287, 4535, 6064], dtype=object)

In [63]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming node_embedding['resume'] and node_embedding['recruitment'] are NumPy arrays
resume_embeddings = node_embedding['resume']
recruitment_embeddings = node_embedding['recruitment']

resume_embeddings_normalized = resume_embeddings / torch.norm(resume_embeddings, dim=1, keepdim=True)
recruitment_embeddings_normalized = recruitment_embeddings / torch.norm(recruitment_embeddings, dim=1, keepdim=True)
top_recruitment_for_resume = []
# Iterate over each resume node
for i in range(resume_embeddings_normalized.shape[0]):
    # Compute cosine similarity between the current resume node and all recruitment nodes
    if i%100 ==0:
      print(i)
    similarity_scores = torch.matmul(recruitment_embeddings_normalized, resume_embeddings_normalized[i])

    # Find the indices of the top 5 most similar recruitment nodes
    top_recruitment_indices = torch.argsort(similarity_scores, descending=True)[:10]
    top_recruitment_indices = [idx.detach().cpu().numpy() for idx in top_recruitment_indices if idx not in node_relation[node_relation['resume_seq']==i]['recruitment_seq'].values][:5]
    # Append the top recruitment indices to the list
    top_recruitment_for_resume.append(top_recruitment_indices) #

# Print the top 5 recruitment nodes for each resume node
for i, top_recruitments in enumerate(top_recruitment_for_resume):
    print(f"Top 5 recruitment nodes most similar to resume node {i}: {top_recruitments}")
    if i == 5 :
      break


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
Top 5 recruitment nodes most similar to resume node 0: [array(3525), array(4431), array(3640), array(5938), array(3358)]
Top 5 recruitment nodes most similar to resume node 1: [array(5471), array(4587), array(5138), array(3195), array(2244)]
Top 5 recruitment nodes most similar to resume node 2: [array(1030), array(1885), array(12), array(2694), array(5576)]
Top 5 recruitment nodes most similar to resume node 3: [array(2877), array(1830), array(2371), array(3624), array(5345)]
Top 5 recruitment nodes most similar to resume node 4: [array(2141), array(1614), array(2236), array(3906

In [45]:
pd.read_csv('sample_submission.csv')

,resume_seq,recruitment_seq
0,U00001,R03134
1,U00001,R00465
2,U00001,R01931
3,U00001,R01607
4,U00001,R06059
...,...,...
42405,U08482,R03134
42406,U08482,R00465
42407,U08482,R01931
42408,U08482,R01607


In [64]:
final_data = []

# Iterate over each resume node and its top 5 recruitment nodes
for i, top_recruitments in enumerate(top_recruitment_for_resume):
    for recruitment_index in top_recruitments:
        # Create a row with resume_seq and recruitment_seq
        row = {'resume_seq': f'U{i+1:05d}', 'recruitment_seq': f'R{recruitment_index+1:05d}'}
        final_data.append(row)

# Create a DataFrame from the list of rows
result_df = pd.DataFrame(final_data)

In [73]:
result_df

,resume_seq,recruitment_seq
0,U00001,R03526
1,U00001,R04432
2,U00001,R03641
3,U00001,R05939
4,U00001,R03359
...,...,...
42405,U08482,R03873
42406,U08482,R02142
42407,U08482,R05407
42408,U08482,R01677


In [71]:
result_df.to_csv('result.csv', index=False)

In [72]:
pd.read_csv('result.csv')

,resume_seq,recruitment_seq
0,U00001,R03526
1,U00001,R04432
2,U00001,R03641
3,U00001,R05939
4,U00001,R03359
...,...,...
42405,U08482,R03873
42406,U08482,R02142
42407,U08482,R05407
42408,U08482,R01677


# GHCF